In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType, DateType
from pyspark.sql.functions import col, concat
from pyspark.sql.functions import to_timestamp, lit, current_timestamp

In [0]:
# CircuitId is a primary key
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                     StructField("year", IntegerType(), True),
                                     StructField("round", IntegerType(), True),
                                     StructField("circuitId", IntegerType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("date", DateType(), True),
                                     StructField("time", StringType(), True),
                                     StructField("url", StringType(), True)
])

In [0]:
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv("dbfs:/mnt/formula1dl612/raw/races.csv")
display(races_df.head(5))

raceId,year,round,circuitId,name,date,time,url
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Grand_Prix
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Grand_Prix
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Grand_Prix
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Grand_Prix


In [0]:
races_selected_df = races_df.select("raceId", "year", "round", "circuitId", "name", "time", "date")
display(races_selected_df.head(5))

raceId,year,round,circuitId,name,time,date
1,2009,1,1,Australian Grand Prix,06:00:00,2009-03-29
2,2009,2,2,Malaysian Grand Prix,09:00:00,2009-04-05
3,2009,3,17,Chinese Grand Prix,07:00:00,2009-04-19
4,2009,4,3,Bahrain Grand Prix,12:00:00,2009-04-26
5,2009,5,4,Spanish Grand Prix,12:00:00,2009-05-10


In [0]:
# lit(' ')
races_final_df = races_selected_df.withColumn("ingestion_date", current_timestamp()) \
.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))
display(races_final_df.head(5))

raceId,year,round,circuitId,name,time,date,ingestion_date,race_timestamp
1,2009,1,1,Australian Grand Prix,06:00:00,2009-03-29,2024-02-23T14:44:58.891+0000,2009-03-29T06:00:00.000+0000
2,2009,2,2,Malaysian Grand Prix,09:00:00,2009-04-05,2024-02-23T14:44:58.891+0000,2009-04-05T09:00:00.000+0000
3,2009,3,17,Chinese Grand Prix,07:00:00,2009-04-19,2024-02-23T14:44:58.891+0000,2009-04-19T07:00:00.000+0000
4,2009,4,3,Bahrain Grand Prix,12:00:00,2009-04-26,2024-02-23T14:44:58.891+0000,2009-04-26T12:00:00.000+0000
5,2009,5,4,Spanish Grand Prix,12:00:00,2009-05-10,2024-02-23T14:44:58.891+0000,2009-05-10T12:00:00.000+0000


In [0]:
races_selected_df = races_final_df.select(col("raceId").alias('race_id'),
                                          col("year").alias('race_year'),
                                          col("round"),
                                          col("circuitId").alias("circuit_id"),
                                          col("name"),
                                          col("ingestion_date"),
                                          col("race_timestamp"))
display(races_selected_df.head(5))

race_id,race_year,round,circuit_id,name,ingestion_date,race_timestamp
1,2009,1,1,Australian Grand Prix,2024-02-23T14:44:59.542+0000,2009-03-29T06:00:00.000+0000
2,2009,2,2,Malaysian Grand Prix,2024-02-23T14:44:59.542+0000,2009-04-05T09:00:00.000+0000
3,2009,3,17,Chinese Grand Prix,2024-02-23T14:44:59.542+0000,2009-04-19T07:00:00.000+0000
4,2009,4,3,Bahrain Grand Prix,2024-02-23T14:44:59.542+0000,2009-04-26T12:00:00.000+0000
5,2009,5,4,Spanish Grand Prix,2024-02-23T14:44:59.542+0000,2009-05-10T12:00:00.000+0000


##### Partitioning

In [0]:
# races_selected_df.write.mode("overwrite").parquet("/mnt/formula1dl612/processed/races")
races_selected_df.write.mode("overwrite").partitionBy('race_year').parquet("/mnt/formula1dl612/processed/races")

In [0]:
%fs
ls /mnt/formula1dl612/processed/races

path,name,size,modificationTime
dbfs:/mnt/formula1dl612/processed/races/_SUCCESS,_SUCCESS,0,1708699518000
dbfs:/mnt/formula1dl612/processed/races/_committed_2088282650369492340,_committed_2088282650369492340,123,1708645916000
dbfs:/mnt/formula1dl612/processed/races/_committed_2525315113697676533,_committed_2525315113697676533,122,1708698589000
dbfs:/mnt/formula1dl612/processed/races/_committed_6875742868824890401,_committed_6875742868824890401,220,1708698518000
dbfs:/mnt/formula1dl612/processed/races/_committed_8683413416780842440,_committed_8683413416780842440,232,1708645996000
dbfs:/mnt/formula1dl612/processed/races/_committed_vacuum4214368763001182036,_committed_vacuum4214368763001182036,96,1708698518000
dbfs:/mnt/formula1dl612/processed/races/_started_6875742868824890401,_started_6875742868824890401,0,1708698516000
dbfs:/mnt/formula1dl612/processed/races/race_year=1950/,race_year=1950/,0,1708698572000
dbfs:/mnt/formula1dl612/processed/races/race_year=1951/,race_year=1951/,0,1708698572000
dbfs:/mnt/formula1dl612/processed/races/race_year=1952/,race_year=1952/,0,1708698572000


In [0]:
display(spark.read.parquet('/mnt/formula1dl612/processed/races').head(5))

race_id,round,circuit_id,name,ingestion_date,race_timestamp,race_year
1053,2,21,Emilia Romagna Grand Prix,2024-02-23T14:45:00.198+0000,2021-04-18T13:00:00.000+0000,2021
1052,1,3,Bahrain Grand Prix,2024-02-23T14:45:00.198+0000,2021-03-28T15:00:00.000+0000,2021
1051,21,1,Australian Grand Prix,2024-02-23T14:45:00.198+0000,2021-11-21T06:00:00.000+0000,2021
1054,3,20,TBC,2024-02-23T14:45:00.198+0000,null,2021
1055,4,4,Spanish Grand Prix,2024-02-23T14:45:00.198+0000,2021-05-09T13:00:00.000+0000,2021
